In [1]:
!pip install ccxt
import ccxt
import pandas as pd
import time
import datetime

# Create a Bitstamp exchange instance with rate limit enabled.
exchange = ccxt.bitstamp({
    'enableRateLimit': True,
})

# Define the symbol, timeframe, and date range.
# Adjust the dates as needed; here we use a historical 6-month period.
symbol = 'BTC/USD'
timeframe = '1d'
start_date = '2025-01-01T00:00:00Z'
end_date   = '2025-02-25T00:00:00Z'

# Convert the dates to millisecond timestamps.
since = exchange.parse8601(start_date)
end_time = exchange.parse8601(end_date)

all_candles = []
print(f"Downloading BTC data from {start_date} to {end_date} on Bitstamp")

# Fetch data in batches until the end of the period is reached.
while since < end_time:
    current_dt = datetime.datetime.utcfromtimestamp(since / 1000).strftime('%Y-%m-%d %H:%M:%S')
    print(f"Fetching data from {current_dt} UTC")
    
    try:
        candles = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=1000)
    except Exception as e:
        print("Error fetching data:", e)
        time.sleep(10)
        continue
    
    if not candles:
        print("No more data returned; exiting loop.")
        break
    
    all_candles.extend(candles)
    last_timestamp = candles[-1][0]
    
    if last_timestamp >= end_time:
        break

    # Advance 'since' by one candle period (5 minutes in milliseconds)
    since = last_timestamp + 5 * 60 * 1000
    time.sleep(exchange.rateLimit / 1000)  # Respect Bitstamp's rate limit

# Convert the list of candles to a Pandas DataFrame.
df = pd.DataFrame(all_candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df[['datetime', 'open', 'high', 'low', 'close', 'volume']]

# Save the DataFrame to a CSV file.
output_file = "/notebooks/future.csv"
df.to_csv(output_file, index=False)
print(f"Downloaded data saved to {output_file}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 29.9 MB/s eta 0:00:00
Fetching data from 2025-01-01 00:00:00 UTC
Downloaded data saved to /notebooks/future.csv
